In [1]:
from helpers import load_all_datasets, compose, str_rep_vdb, str_rep_no_steps, CBRDatabase, str_rep_vdb_ing, str_rep_vdb_nam 
import pandas as pd

additional_cbr_1, additional_cbr_2, additional_validation, cbr_augmentation, cbr_database, model_training, validation = load_all_datasets()

cbr_database = cbr_database.set_index("id")

EX_APP = "f"

INITIAL_C = 10
GENERATED_C = 2

DATABASE_NAME = f"cbr_database_{INITIAL_C}_{GENERATED_C}_{EX_APP}"
OUTPUT_FILE = f"cbr_{INITIAL_C}_{GENERATED_C}_{EX_APP}_gpt2_input"

IX_FILE = "cbr_augmentation_12_gpt2_results_p_ix_f.csv"
IX_FILE2 = "cbr_augmentation_gpt2_results_p_ix_f.csv"
dfix1 = pd.read_csv(f"{IX_FILE}")
dfix2 = pd.read_csv(f"{IX_FILE2}")

dfix = pd.concat([dfix1, dfix2])
print(len(dfix))


17835


In [2]:
# cbr_db = pd.concat([cbr_database, additional_cbr_1])
cbr_database = cbr_database[:int(INITIAL_C*1000)]
dfix = dfix[:int(GENERATED_C*1000)]
dfix = dfix.set_index("id")
cbr_db_ix = pd.concat([cbr_database, dfix])
# cbr_db_ix = cbr_db_ix.set_index("id")

cbr_reps = str_rep_vdb(cbr_db_ix)
len(cbr_reps)

12000

In [3]:
meta = CBRDatabase.create_metadatas([int(e) for e in list(cbr_db_ix.index.values)], {"dataset":DATABASE_NAME})
vdb = CBRDatabase(cbr_reps, metadatas=meta, db_directory=f"dataset_vector/{DATABASE_NAME}", initialize_new=True)

# vdb = CBRDatabase(db_directory=f"dataset_vector/{DATABASE_NAME}", initialize_new=False)

print(len(vdb.get_texts()))
print(vdb.similarity_search(query="test", k=2))

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522
12000
[Document(page_content='name: desperation cookies ; ingredients: butter, white sugar, brown sugar, vanilla, baking soda, salt, eggs, flour, chocolate chips, nuts ; preparation:', metadata={'dataset': 'cbr_database_10_2_f', 'recipe_id': 320579}), Document(page_content='name: splenda spice cookies ; ingredients: egg beaters egg substitute, splenda brown sugar blend, ginger, cinnamon, allspice, pepper, flour, salt, baking soda ; preparation:', metadata={'dataset': 'cbr_database_10_2_f', 'recipe_id': 348303})]


In [4]:
# tem = vdb.similarity_search(query="akjsjd", k=5)
# recipe_ixs = [e.metadata[CBRDatabase.RECIPE_ID_KEY] for e in tem]
# recipe_ixs

In [5]:
# generowanie inputów - 850 tokenów 
from pprint import pprint
import random
from transformers import LlamaTokenizerFast

tokenizer = LlamaTokenizerFast.from_pretrained("hf-internal-testing/llama-tokenizer")

inputs = []
pe_recipes = []
target_recipe = []
lens = []

for i, v_recipe in validation.iterrows():
    while True:
        tem = vdb.similarity_search(query=str_rep_no_steps(v_recipe), k=5)
        recipe_ixs = [e.metadata[CBRDatabase.RECIPE_ID_KEY] for e in tem]
        
        #dev
        # print(tem)
        # composed = compose(cbr_db_ix.loc[recipe_ixs[0:2]], v_recipe)
        # print(recipe_ixs)
        # print(composed)
        # pprint(composed.split("name: "))
        # raise
        #dev

        gen_input = compose(cbr_db_ix.loc[recipe_ixs], v_recipe)
        br = False
        if len(tokenizer.encode(gen_input)) <= 850: 
            br = True
        else:
            for ii in list(range(4, -1, -1)):
                gen_input = compose(cbr_db_ix.loc[recipe_ixs[:ii] + recipe_ixs[ii+1:]], v_recipe)
                if len(tokenizer.encode(gen_input)) <= 850: 
                    br = True
                    print(f"1 - {len(tokenizer.encode(gen_input))}")
                    break
            if br: break
            for ii in list(range(3, -1, -1)):
                gen_input = compose(cbr_db_ix.loc[recipe_ixs[:ii] + recipe_ixs[ii+2:]], v_recipe)
                if len(tokenizer.encode(gen_input)) <= 850: 
                    br = True
                    print(f"2 - {len(tokenizer.encode(gen_input))}")
                    break
            if br: break
            for ii in list(range(2, -1, -1)):
                gen_input = compose(cbr_db_ix.loc[recipe_ixs[:ii] + recipe_ixs[ii+3:]], v_recipe)
                if len(tokenizer.encode(gen_input)) <= 850: 
                    br = True
                    print(f"2 - {len(tokenizer.encode(gen_input))}")
                    break
        
        if not br:
            print(cbr_db_ix.loc[recipe_ixs].steps.values)
            print(f"REPEAT - {len(tokenizer.encode(gen_input))}")
        else:
            lens.append(len(tokenizer.encode(gen_input)))
            break
            
        
    pe_recipes.append(recipe_ixs)
    target_recipe.append(v_recipe["id"])
    inputs.append(gen_input)

/home/maciej/anaconda3/envs/ml_lm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1 - 802
1 - 811
1 - 752
1 - 757
1 - 767
1 - 723
1 - 804
2 - 646
1 - 629
1 - 822
1 - 738
1 - 758
1 - 831
1 - 706
1 - 681
1 - 747
1 - 698
1 - 812
1 - 800
1 - 749
1 - 850
1 - 820
1 - 751
1 - 832
1 - 707
1 - 761
1 - 656
1 - 844
1 - 839
1 - 777
1 - 743
1 - 725
1 - 665
1 - 662
1 - 756
1 - 746
1 - 743
1 - 753
1 - 747
1 - 847
1 - 809
1 - 816
1 - 848
1 - 833
1 - 710
1 - 680
1 - 720
1 - 683
1 - 818
1 - 759
1 - 828
1 - 768
1 - 742
1 - 758
1 - 822
1 - 793
1 - 758
1 - 729
1 - 729
1 - 749
1 - 700
2 - 646
1 - 810
1 - 823
1 - 771
1 - 725
1 - 629
1 - 711
1 - 739
1 - 811
1 - 772
1 - 786
1 - 799
2 - 690
1 - 700
1 - 703
1 - 707
1 - 701
1 - 712
1 - 688
1 - 721
1 - 608
1 - 779
1 - 743
1 - 763
1 - 768
1 - 832
1 - 735
1 - 782
1 - 749
1 - 733
1 - 705
1 - 847
1 - 696
1 - 675
1 - 658
1 - 843
1 - 747
1 - 784
1 - 781
1 - 763
1 - 756
2 - 717
2 - 701
1 - 832
1 - 722
1 - 759
1 - 770
1 - 679
1 - 810
1 - 770
1 - 770
1 - 729
1 - 772
1 - 793
1 - 820
1 - 826
1 - 746
1 - 848
1 - 684
1 - 755
1 - 836
1 - 690
1 - 797
1 - 766


In [6]:
df = pd.DataFrame()
df["input"] = inputs
df["pe_recipes"] = pe_recipes
df["target_recipe"] = target_recipe
df.to_csv(f"input_datasets/{OUTPUT_FILE}.csv", index=False)

In [7]:
vdb.similarity_search(query="str_rep_no_steps(v_recipe)", k=4)

[Document(page_content='name: v s crock pot bottom round roast ; ingredients: boneless bottom round roast, dijon mustard, brown sugar, worcestershire sauce, dry red wine, beef broth, onion, carrots, kosher salt & freshly ground black pepper ; preparation:', metadata={'dataset': 'cbr_database_10_2_f', 'recipe_id': 276903}),
 Document(page_content='name: hunter s delight ; ingredients: bacon, red potatoes, onions, venison steak, cream-style corn, worcestershire sauce, sugar, seasoning salt ; preparation:', metadata={'dataset': 'cbr_database_10_2_f', 'recipe_id': 331054}),
 Document(page_content='name: mock crock pot roast ; ingredients: beef eye round, root beer, garlic powder, lemon pepper, cornstarch, water ; preparation:', metadata={'dataset': 'cbr_database_10_2_f', 'recipe_id': 213475}),
 Document(page_content='name: savory slow cooker beef roast ; ingredients: boneless beef roast, onion, vegetable juice, brown gravy mix, garlic cloves, pepper, thyme ; preparation:', metadata={'datas

In [8]:
from collections import Counter


Counter(df.input.apply(lambda x: x.split("name:")).apply(lambda x: len(x)))

Counter({6: 1085, 7: 1008, 5: 32})